## Import packages

In [ ]:
import os, sys

if sys.platform == "darwin":
    os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

if not "root_dir" in globals():
    now_dir = os.getcwd() # skip examples/ipynb
    root_dir = os.path.join(now_dir, "../../")
    sys.path.append(root_dir)
    print("init root dir to", root_dir)

from dotenv import load_dotenv
load_dotenv(os.path.join(root_dir, "sha256.env"))

import torch
torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision('high')

import ChatTTS
from tools.logger import get_logger
from IPython.display import Audio

## Load Models

In [ ]:
os.chdir(root_dir)

chat = ChatTTS.Chat(get_logger("ChatTTS"))

### Here are three choices for loading models:

#### 1. Load models from Hugging Face:

In [ ]:
# use force_redownload=True if the weights have been updated.
chat.load_models(source='huggingface', force_redownload=True)

#### 2. Load models from local directories 'asset' and 'config':

In [ ]:
chat.load_models()
# chat.load_models(source='local') same as above

#### 3. Load models from a custom path:

In [ ]:
# write the model path into custom_path
chat.load_models(source='custom', custom_path='YOUR CUSTOM PATH')

## Inference

### Batch infer

In [ ]:
texts = ["So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with.",]*3 \
        + ["我觉得像我们这些写程序的人，他，我觉得多多少少可能会对开源有一种情怀在吧我觉得开源是一个很好的形式。现在其实最先进的技术掌握在一些公司的手里的话，就他们并不会轻易的开放给所有的人用。"]*3     

wavs = chat.infer(texts)

In [ ]:
Audio(wavs[0], rate=24_000, autoplay=True)

In [ ]:
Audio(wavs[3], rate=24_000, autoplay=True)

### Custom params

In [ ]:
params_infer_code = {'prompt':'[speed_5]', 'temperature':.3}
params_refine_text = {'prompt':'[oral_2][laugh_0][break_6]'}

wav = chat.infer('四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片等，每样都让人垂涎三尺。', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

### Fix random speaker

In [ ]:
rand_spk = chat.sample_random_speaker()
params_infer_code = {'spk_emb' : rand_spk, }

wav = chat.infer('四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

### Two stage control

In [ ]:
text = "So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with."
chat.infer(text, refine_text_only=True)

In [ ]:
text = 'so we found being competitive and collaborative [uv_break] was a huge way of staying [uv_break] motivated towards our goals, [uv_break] so [uv_break] one person to call [uv_break] when you fall off, [uv_break] one person who [uv_break] gets you back [uv_break] on then [uv_break] one person [uv_break] to actually do the activity with.'
wav = chat.infer(text, skip_refine_text=True)

## LLM Call

In [ ]:
from ChatTTS.experimental.llm import llm_api

API_KEY = ''
client = llm_api(api_key=API_KEY,
        base_url="https://api.deepseek.com",
        model="deepseek-chat")

In [ ]:
user_question = '四川有哪些好吃的美食呢?'
text = client.call(user_question, prompt_version = 'deepseek')
print(text)
text = client.call(text, prompt_version = 'deepseek_TN')
print(text)

In [ ]:
params_infer_code = {'spk_emb' : rand_spk, 'temperature':.3}

wav = chat.infer(text, params_infer_code=params_infer_code)